In [35]:
import os
from os import path
import glob
import pandas as pd
import json
from collections import defaultdict

In [36]:
perf_dir = "/home/shtoshni/Research/events/models/perf"
# perf_dir = "/share/data/speech/shtoshni/research/events/models/perf"
slurm_id_list = ["6410467"]  #"6407936"

files = []

for slurm_id in slurm_id_list:
    files.extend(sorted(glob.glob(path.join(perf_dir, slurm_id + "*")), 
                                  key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1])))
    
# files = sorted(files, key=lambda x: int(path.splitext(path.basename(x))[0].split('_')[1]))
print(len(files))

58


In [37]:
model_dict_list = []
for file in files:
    model_dict_list.append(json.loads(open(file).read()))

In [38]:
def determine_varying_attributes(model_dict_list, ignore_attribs=['model_dir', 'train', 'test', 'dev', 'pretrained_mention_model',
                                                                  'slurm_id', 'best_model_dir', 'data_dir']):
    attrib_to_vals = defaultdict(set)
    for model_dict in model_dict_list:
        for attrib, val in model_dict.items():
            if attrib in ignore_attribs:
                continue
            else:
                attrib_to_vals[attrib].add(val)
    
    
    varying_attribs = []
    for attrib, vals in attrib_to_vals.items():
        if len(vals) > 1:
            varying_attribs.append(attrib)
            
    return varying_attribs

In [39]:
varying_attribs = determine_varying_attributes(model_dict_list)


In [40]:
perf_df = pd.DataFrame(columns=(varying_attribs))

for model_dict in model_dict_list:
    perf_dict = {}
    for attrib in varying_attribs:
        perf_dict[attrib] = model_dict[attrib]
            
    perf_dict['fscore'] = round(model_dict['dev']['fscore'], 1)
    perf_dict['test_f'] = round(model_dict['test']['fscore'], 1)
                
    perf_df = perf_df.append(perf_dict, ignore_index=True)

In [41]:
idx = perf_df.groupby(['srl_loss_wt', 'seed'])['fscore'].idxmax()
dev_max = perf_df.iloc[idx]
dev_max

,sample_invalid,srl_loss_wt,srl_loss_type,seed,fscore,test_f
35,0.50,0.1,diff,5,48.7,48.0
6,0.25,0.1,diff,6,49.0,50.1
2,0.25,0.1,cross_entropy,7,49.7,49.0
33,0.50,0.1,cross_entropy,8,48.5,49.0
9,0.25,0.1,diff,9,49.3,49.5
43,0.50,0.5,diff,5,48.6,47.6
11,0.25,0.5,cross_entropy,6,48.0,48.6
40,0.50,0.5,cross_entropy,7,49.4,47.9
18,0.25,0.5,diff,8,48.7,49.0
14,0.25,0.5,cross_entropy,9,49.2,48.8


In [43]:
perf_df.groupby(['srl_loss_wt', 'sample_invalid', 'srl_loss_type' ])[['fscore', 'test_f']].mean()


fscore     test_f
srl_loss_wt sample_invalid srl_loss_type                   
0.1         0.25           cross_entropy   48.30  49.000000
                           diff            48.46  49.460000
            0.50           cross_entropy   47.90  48.880000
                           diff            47.70  48.433333
0.5         0.25           cross_entropy   48.14  48.600000
                           diff            48.12  48.960000
            0.50           cross_entropy   48.08  48.220000
                           diff            47.76  48.260000
1.0         0.25           cross_entropy   47.52  48.220000
                           diff            47.74  48.400000
            0.50           cross_entropy   47.76  48.400000
                           diff            48.14  48.620000